In [1]:
# Импорт всех модулей, думаю тут по азваниям все понятно
# Во всех, кроме electric заглушки
# Ну и модули это файлы .py
import optic
import thermal
import degrading
import electric

import numpy as np
import matplotlib.pyplot as plt

In [3]:
"""
Решаем один временной шаг.
    Рассчет интенсивностей и источников тепла из оптики.
    Рассчет потенциалов.
    Рассчет новых температур.
    Рассчет деградации.
    
Тут первое на что обратить внимание это формат функций, которые тебе нужно будет написать.

optic.solve(cnf, properties, old_T, old_g, bound_optic)
thermal.solve(cnf, properties, old_T, new_Q, bound_thermal)

cnf  - словарь в котором лежат все размеры задачи (можешь ниже промотать, там он есть)
old_T, old_g - очевидно думаю температура и степень деградации на предыдущем шаге, от них зависят свойства
properties - словарь со свойствами биоткани (тоже ниже подробнее) - функционально заданых! 
не в каждой точне, а просто зависимость от температуры и степени деградации

bound_optic, bound_thermal - инфа о граничных условиях, тоже ниже подробнее
"""
def step_solver(cnf, properties, old_T, old_g, bound_optic, bound_electric, bound_thermal):
    new_I, new_Q = optic.solve(cnf, properties, old_T, old_g, bound_optic)
    new_u, new_q = electric.solve(cnf, properties, old_T, old_g, bound_electric)
    
    new_T = thermal.solve(cnf, properties, old_T, new_Q, bound_thermal)
    new_g = degrading.solve(cnf, properties, old_g, new_T)
    
    return {"I": new_I, "Q": new_Q, "u": new_u, "q": new_q, "T": new_T, "g": new_g}

In [4]:
"""
Теперь уже несколько временных шагов.
Забиваем значениями переменных матрицу solution.
"""
def solver(cnf, properties, condition_start, bound_optic, bound_electric, bound_thermal):
    solution = {
        "T": np.array([condition_start['T']]),
        "g": np.array([condition_start['g']])
    }
    for k in range(K):
        step_solution = step_solver(cnf, properties, solution['T'][-1], solution['g'][-1], 
                                    bound_optic, bound_electric, bound_thermal)
    
        solution['I'] = np.concatenate((solution['I'], [step_solution['I']]))
        solution['Q'] = np.concatenate((solution['Q'], [step_solution['Q']]))
        solution['u'] = np.concatenate((solution['u'], [step_solution['u']]))
        solution['q'] = np.concatenate((solution['q'], [step_solution['q']]))
        solution['T'] = np.concatenate((solution['T'], [step_solution['T']]))
        solution['g'] = np.concatenate((solution['g'], [step_solution['g']]))

In [5]:
"""
Словарь с количеством клеток и шагом.
Для всех задач потенциально сетка одна.
Кроме моей, там еще воздух добавляется, но в ткани сетка та же.
"""
cnf = {
    "N": 100,
    "dn": 10 ** (-9),
    "M": 200,
    "dm": 10 ** (-9),
    "K" : 10,
    "dK": 10 ** (-6),
    
    "NAir": 100
}

In [6]:
"""
Словарь свойств.
Каждое свойство зависит от температуры.
Хочешь свойства при данной температуре по всей сетке - просто тыкаешь матрицу температур
properties['eps']([[1,2],[3,4]],[[1,1],[1,1]])
Физ.Константы тоже здесь
"""

properties = {
    "eps": lambda T,g: 81 * T/T,
    "sigma": lambda T,g: (10 ** (-1) * (1-g) + 10 ** (-3) * g) * (1-0.01 * (T-360)),
    "diffusion": lambda T,g: 10 ** (-7) * T,
    
    'eps0': 8.85 * 10 ** (-12)
}

In [7]:
"""
N,M - локальные, в твои функции уйдет cnf, там уже сам достанешь, будут у тебя свои N,M внутри функции
"""
M = cnf['M']
N = cnf['N']
NAir = cnf['NAir']

"""
С начальными условиями все вроде просто
g = 1 - недеградированная ткань
g = 0 - полностью деградированная будет
"""
condition_start = {
    'T': 300 * np.ones((N,M)),
    'g': 1 * np.ones((N,M))
}

"""
В классе Boundary храниться самая общая инфа о граничных условиях.
Для какой переменной условие - variable_name
Тип граничного условия (Дирихле, Ньютон-Рихман итд) - condition_type
Координаты точке в которых ты пишешь гран условие - coordinates - т.е. можно смело 
задавать условие не только на всей границе верхней, но и на кусках (мне вот нужно будет с электродами)
external_normal - внешняя нормаль, т.е. для n=0 - [-1,0], это упращает запись условий с производной, 
можешь глянуть как я реализовал Неймана
coefficients - собственно некая цифирка, значение переменной на границе или производной или еще чего

Ниже пример:
"""
class Boundary():
    def __init__(self,variable_name, condition_type, coordinates, external_normal, coefficients):
        self.variable_name = variable_name
        self.condition_type = condition_type
        self.coefficients = coefficients
        self.coordinates = coordinates
        self.external_normal = external_normal


coo_n_start = np.array([[0, m] for m in range(M)])
coo_n_stop = np.array([[N - 1, m] for m in range(M)])
coo_m_start = np.array([[n, 0] for n in range(0, N)])
coo_m_stop = np.array([[n, M - 1] for n in range(0, N)])

"""
количество координат должно совпадать с количеством коэффициентов, остальное все вроде очевидно
Тут еще нюанс, гран условия тоже функции от T,g ибо в них коэффициенты могут выражаться через свойства, 
может где встретиться у тебя
"""
def condition_boundary_termal(T,g):
    return [
        Boundary('T','Newton',coo_n_start,[-1,0],np.ones(M)),
        Boundary('T','Dirichlet',coo_n_stop,[1,0],np.zeros(M)),
        Boundary('T','Newmann',coo_m_start[1:-1],[0,-1],np.zeros(N-2)),
        Boundary('T','Dirichlet',coo_m_stop[1:-1],[0,1],np.zeros(N-2)),
    ]

def condition_boundary_optic(T,g):
    return [
        Boundary('I','Newton',coo_n_start,[-1,0],np.ones(M)),
        Boundary('I','Dirichlet',coo_n_stop,[1,0],np.zeros(M)),
        Boundary('I','Newmann',coo_m_start[1:-1],[0,-1],np.zeros(N-2)),
        Boundary('I','Dirichlet',coo_m_stop[1:-1],[0,1],np.zeros(N-2)),
    ]

"""
В электричестве условий больше в 1,5 раза примерно, а это я еще электроды не добавлял...
"""

coo_air_n_start = np.array([[0, m] for m in range(M)])
coo_air_n_stop = np.array([[NAir - 1, m] for m in range(M)])
coo_air_m_start = np.array([[n, 0] for n in range(0, NAir)])
coo_air_m_stop = np.array([[n, M - 1] for n in range(0, NAir)])

coo_tissue_n_start = np.array([[NAir, m] for m in range(M)])
coo_tissue_n_stop = np.array([[N + NAir - 1, m] for m in range(M)])
coo_tissue_m_start = np.array([[n, 0] for n in range(NAir, N + NAir)])
coo_tissue_m_stop = np.array([[n, M - 1] for n in range(NAir, N + NAir)])

# el1_M = int(7 * M / 15)
# el2_M = int(8 * M / 15)
# ne_M = np.delete(np.arange(M),[el1_M,el2_M])
# coo_air_el1 = np.array([[NAir - 1, el1_M]])
# coo_air_el2 = np.array([[NAir - 1, el2_M]])
# coo_air_n_stop_ne = np.array([[NAir - 1, m] for m in ne_M])  
# coo_tissue_el1 = np.array([[NAir, el1_M]])
# coo_tissue_el2 = np.array([[NAir, el2_M]])
# coo_tissue_n_start_ne = np.array([[NAir, m] for m in ne_M])  

def bound_electric(T,g): 
    return [
        Boundary('u','Dirichlet',coo_air_n_start,[-1,0],np.zeros(M)),
        Boundary('u','Dirichlet',coo_air_m_start[1:-1],[0,-1],np.zeros(NAir-2)),
        Boundary('u','Dirichlet',coo_air_m_stop[1:-1],[0,1],np.zeros(NAir-2)),

        Boundary('q','Dirichlet',coo_air_n_start,[-1,0],np.zeros(M)),
        Boundary('q','Dirichlet',coo_air_m_start[1:-1],[0,-1],np.zeros(NAir-2)),
        Boundary('q','Dirichlet',coo_air_m_stop[1:-1],[0,1],np.zeros(NAir-2)),

        Boundary('u','Newmann',coo_tissue_n_stop,[1,0],np.zeros(M)),
        Boundary('u','Dirichlet',coo_tissue_m_start[1:-1],[0,-1],np.ones(N-2)),
        Boundary('u','Dirichlet',coo_tissue_m_stop[1:-1],[0,1],-np.ones(N-2)),

        Boundary('j','Newmann',coo_tissue_n_stop,[1,0],np.zeros(M)),
        Boundary('j','Newmann',coo_tissue_m_start[1:-1],[0,-1],np.zeros(N-2)),
        Boundary('j','Newmann',coo_tissue_m_stop[1:-1],[0,1],np.zeros(N-2)),

        Boundary('u','Continiosly',[coo_air_n_stop,coo_tissue_n_start],[[0,1],[0,-1]],
                 [np.ones(M),properties['eps'](T,g)]),
        Boundary('v','Dirichlet',coo_air_n_stop,[0,1],np.zeros(M)),
        Boundary('j','Newmann',coo_tissue_n_start,[-1,0],np.zeros(M))
    ]

In [8]:
"""
Демонстрация тупой ошибки жупитера из-за которой я буду сейчас все переносить таки на нормальный питон
"""
u,v = electric.solve(cnf, properties, condition_start['T'], condition_start['g'], bound_electric)

TypeError: cannot unpack non-iterable NoneType object

In [9]:
electric.Dirichlet(cnf, "u", [[1,1],[2,2]], [2,3])

(<2x80000 sparse matrix of type '<class 'numpy.float64'>'
 	with 2 stored elements in COOrdinate format>, [2, 3])

In [10]:
a,b = electric.get_bound_matrix(cnf, bound_electric(condition_start['T'], condition_start['g'])[0])